In [ ]:
import sys

sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import data.drawings.make_tasks as drawing_tasks
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from src.analysis_utilities import IterativeExperimentAnalyzer
from src.config_builder import ExperimentType

In [ ]:
%config InlineBackend.figure_format = 'retina'

sns.set_theme(style="whitegrid", font_scale=1.25)

In [ ]:
# EXPERIMENT_NAME = "gg_drawings"

# DOMAIN_NAMES_CAMERA = {
#     "drawings_nuts_bolts": "nuts & bolts",
#     "drawings_wheels": "vehicles",
#     "drawings_dials": "gadgets",
#     "drawings_furniture": "furniture",
# }

In [ ]:
EXPERIMENT_NAME = "gg_laps_domains"

DOMAIN_NAMES_CAMERA = {
    # "re2_no_other_util": "REGEX (--no-other-util)",
    "re2": "REGEX",
    "clevr": "CLEVR",
}

In [ ]:
FIGURES_DIR = os.path.join("figures", EXPERIMENT_NAME)
os.makedirs(FIGURES_DIR, exist_ok=True)

# Multi-domain analysis

In [ ]:
analyzer = IterativeExperimentAnalyzer(
    experiment_name=EXPERIMENT_NAME,
    allow_incomplete_results=False,
)

In [ ]:
df_list = []
for domain in DOMAIN_NAMES_CAMERA:
    df = analyzer.get_results_for_domain(domain=domain)
    df["domain"] = DOMAIN_NAMES_CAMERA[domain]
    df_list.append(df)
df_domains = pd.concat(df_list, axis=0).reset_index(drop=True)

In [ ]:
df_domains = analyzer.format_dataframe_camera(df_domains)

g = sns.catplot(
    data=df_domains,
    x=analyzer.COL_NAMES_CAMERA["batch_size"],
    y=analyzer.COL_NAMES_CAMERA["description_length"],
    hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
    col="domain",
    col_wrap=2,
    kind="point",
    sharex=False,
    sharey=False,
    legend=False,
    aspect=1.5,
    palette=analyzer.EXPERIMENT_TYPES_PALETTE,
)

g.set_axis_labels(
    analyzer.COL_NAMES_CAMERA["batch_size"],
    analyzer.COL_NAMES_CAMERA["description_length"],
    fontsize=14,
)
g.set_xticklabels(size=12)
g.set_yticklabels(size=12)
g.set_titles(col_template="{col_name}", size=18)

# lgd = plt.legend(bbox_to_anchor=(1.02, df_domains["domain"].nunique() // 2 + 0.35), loc="upper left", fontsize=20)
lgd = plt.legend(bbox_to_anchor=(-1.3, -0.2), loc="upper left", fontsize=20, ncol=3)


plt.savefig(
    os.path.join(FIGURES_DIR, f"{EXPERIMENT_NAME}_results_camera.pdf"),
    dpi=300,
    # bbox_extra_artists=(lgd,),
    bbox_inches="tight",
)

# Programs

In [ ]:
df_codex_by_query = analyzer.get_codex_programs(use_results_by_query=True)

In [ ]:
g = sns.catplot(
    data=analyzer.format_dataframe_camera(df_codex_by_query).query("origin != 'train'"),
    kind="bar",
    col="domain",
    col_wrap=2,
    x=analyzer.COL_NAMES_CAMERA["batch_size"],
    y="valid",
    hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
    palette=analyzer.EXPERIMENT_TYPES_PALETTE,
    sharex=False,
    # sharey=False,
    legend=False,
)

# g.set_xticklabels(size=12)
# g.set_yticklabels(size=12)
g.set_axis_labels(
    analyzer.COL_NAMES_CAMERA["batch_size"],
    "Valid sampled programs (%)",
    fontsize=14,
)
g.set_titles(col_template="{col_name}", size=18)

plt.ylim([0, 1])

lgd = plt.legend(bbox_to_anchor=(-1.2, -0.2), loc="upper left", fontsize=12, ncol=3)

plt.savefig(
    os.path.join(FIGURES_DIR, f"{EXPERIMENT_NAME}_valid_camera.pdf"),
    dpi=300,
    bbox_extra_artists=(lgd,),
    bbox_inches="tight",
)

In [ ]:
df_codex = analyzer.get_codex_programs(use_results_by_query=True)
df_codex.loc[df_codex["origin"] == "train", "experiment_type"] = "train"

In [ ]:
analyzer.EXPERIMENT_TYPES_PALETTE["train"] = analyzer.EXPERIMENT_TYPES_PALETTE[ExperimentType.STITCH]

g = sns.catplot(
    kind="violin",
    col="domain",
    data=analyzer.format_dataframe_camera(df_codex),
    x=analyzer.COL_NAMES_CAMERA["batch_size"],
    y="program_str_len",
    hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
    palette=analyzer.EXPERIMENT_TYPES_PALETTE,
    col_wrap=1,
    aspect=4,
    sharey=False,
    sharex=False, # For laps_domains
    legend=False,
    scale="width"
);

g.set_axis_labels(
    analyzer.COL_NAMES_CAMERA["batch_size"],
    "Program string length",
    fontsize=18,
)
g.set_titles(col_template="{col_name}", size=24)

lgd = plt.legend(bbox_to_anchor=(0, -0.2), loc="upper left", fontsize=20, ncol=4)

plt.savefig(
    os.path.join(FIGURES_DIR, f"{EXPERIMENT_NAME}_string_length_camera.pdf"),
    dpi=300,
    bbox_extra_artists=(lgd,),
    bbox_inches="tight",
)

In [ ]:
# NOTE(gg): Not sure why, but using data from the results_by_query produces numbers that do not track with 
# the actual results. I think this has something to do with how we dedup. In any case, the correct version
# of this plot is below, using the OVERALL results.

# # Darken each color in the palette
# from PIL import ImageColor

# DARKEN_RATIO = 0.6
# PALLETE_DARKENED = {
#     "_" + k: tuple(r * DARKEN_RATIO * 1 / 256 for r in ImageColor.getcolor(hex_str, "RGB"))
#     for k, hex_str in analyzer.EXPERIMENT_TYPES_PALETTE.items()
# }

# fig, ax = plt.subplots(2, 2, figsize=(12, 8))
# ax = ax.flatten()

# for i, domain in enumerate(DOMAIN_NAMES_CAMERA):
#     df_unique_counts = (
#         df_codex_by_query[(df_codex_by_query["domain"] == domain) & (df_codex_by_query["origin"] == "codex")]
#         .drop_duplicates(subset=["program"])
#         .groupby(["experiment_type", "batch_size", "seed"])
#         .sum()
#         .reset_index()
#     )

#     g1 = sns.barplot(
#         ax=ax[i],
#         data=analyzer.format_dataframe_camera(df_unique_counts),
#         x=analyzer.COL_NAMES_CAMERA["batch_size"],
#         y="valid",
#         hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
#         palette=analyzer.EXPERIMENT_TYPES_PALETTE,
#     )
#     sns.despine()
#     g1.legend_.remove()

#     df_unique_counts2 = analyzer.format_dataframe_camera(df_unique_counts)
#     df_unique_counts2[analyzer.COL_NAMES_CAMERA["experiment_type"]] = "_" + df_unique_counts2[analyzer.COL_NAMES_CAMERA["experiment_type"]]
    
#     g2 = sns.barplot(
#         ax=ax[i],
#         data=df_unique_counts2,
#         x=analyzer.COL_NAMES_CAMERA["batch_size"],
#         y="match_train",
#         hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
#         palette=PALLETE_DARKENED,
#         errwidth=2,
#     )
#     sns.despine()
#     g2.legend_.remove()

#     g1.set_title(analyzer.DOMAIN_NAMES_CAMERA[domain])
#     g1.set_ylabel("Unique programs sampled")
    
#     # g1.set_ylim([0, 50])
#     # g2.set_ylim([0, 50])

# plt.subplots_adjust(hspace=0.4)
# lgd = g1.legend(bbox_to_anchor=(-1.3, -0.3), loc="upper left", fontsize=12, ncol=3)

# Using the OVERALL results

In [ ]:
df_codex = analyzer.get_codex_programs(use_results_by_query=False)

In [ ]:
df_codex

In [ ]:
df_codex.valid.isnull().any()

In [ ]:
df_codex["match_prompt"].value_counts()

In [ ]:
for info, df_group in df_codex.groupby(["domain", "experiment_type", "batch_size", "random_seed"]):
    if df_group.duplicated(subset=["hash"]).any():
        raise ValueError(info)

In [ ]:
# Darken each color in the palette
from PIL import ImageColor

DARKEN_RATIO = 0.6
PALLETE_DARKENED = {
    "_" + k: tuple(r * DARKEN_RATIO * 1 / 256 for r in ImageColor.getcolor(hex_str, "RGB"))
    for k, hex_str in analyzer.EXPERIMENT_TYPES_PALETTE.items()
}

fig, ax = plt.subplots(len(DOMAIN_NAMES_CAMERA) // 2, 2, figsize=(12, 2 * len(DOMAIN_NAMES_CAMERA)))
ax = ax.flatten()

for i, domain in enumerate(DOMAIN_NAMES_CAMERA):
    df_unique_counts = (
        df_codex[(df_codex["domain"] == domain)]
        .groupby(["experiment_type", "batch_size", "random_seed"])
        [["valid", "match_prompt"]]
        .sum()
        .reset_index()
    )

    g1 = sns.barplot(
        ax=ax[i],
        data=analyzer.format_dataframe_camera(df_unique_counts),
        x=analyzer.COL_NAMES_CAMERA["batch_size"],
        y="valid",
        hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
        palette=analyzer.EXPERIMENT_TYPES_PALETTE,
    )
    sns.despine()
    g1.legend_.remove()

    df_unique_counts2 = analyzer.format_dataframe_camera(df_unique_counts)
    df_unique_counts2[analyzer.COL_NAMES_CAMERA["experiment_type"]] = "_" + df_unique_counts2[analyzer.COL_NAMES_CAMERA["experiment_type"]]
    
    g2 = sns.barplot(
        ax=ax[i],
        data=df_unique_counts2,
        x=analyzer.COL_NAMES_CAMERA["batch_size"],
        y="match_prompt",
        hue=analyzer.COL_NAMES_CAMERA["experiment_type"],
        palette=PALLETE_DARKENED,
        errwidth=2,
    )
    sns.despine()
    g2.legend_.remove()

    g1.set_title(analyzer.DOMAIN_NAMES_CAMERA[domain])
    g1.set_ylabel("Number of unique programs")
    
    g1.set_ylim([0, 50])
    g2.set_ylim([0, 50])

plt.subplots_adjust(hspace=0.4)
lgd = g1.legend(bbox_to_anchor=(-1.3, -0.3), loc="upper left", fontsize=12, ncol=3)

plt.savefig(
    os.path.join(FIGURES_DIR, f"{EXPERIMENT_NAME}_unique_camera.pdf"),
    dpi=300,
    bbox_extra_artists=(lgd,),
    bbox_inches="tight",
)

In [ ]:
df_unique_counts